In [4]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt
import GPy
import random

import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from sklearn import decomposition
import plotly
import plotly.graph_objects as go
import plotly.express as px

In [5]:
cd /Users/liangqiaohao/Downloads/PV Lab/2020/March/Learning project/3D Printing

/Users/liangqiaohao/Downloads/PV Lab/2020/March/Learning project/3D printing


In [8]:
dataset_3D = pd.read_csv(r'/Users/liangqiaohao/Downloads/PV Lab/2020/March/Learning project/3D printing/parameterspace.csv')
X_3D = dataset_3D.iloc[:,0:5]
# i put negative sign here because default BO is for minimization
X_3D['toughness'] = -dataset_3D.iloc[:,5].values
X_run_3D = X_3D.groupby(['n', 'theta','r','t'])['toughness'].agg(lambda x: x.unique().mean())
X_run_3D = (X_run_3D.to_frame()).reset_index()
s_scaler = preprocessing.StandardScaler()
X_3D_normalized_values = s_scaler.fit_transform(X_run_3D[['n', 'theta','r','t', 'toughness']].values)
X_3D_normalized = pd.DataFrame(X_3D_normalized_values, columns = ['n', 'theta','r','t', 'toughness'])

X_3D_normalized


,n,theta,r,t,toughness
0,-1.341641,-1.53393,-1.565561,-1.224745,1.771761
1,-1.341641,-1.53393,-1.565561,0.000000,1.551493
2,-1.341641,-1.53393,-1.565561,1.224745,1.428859
3,-1.341641,-1.53393,-0.939336,-1.224745,1.503930
4,-1.341641,-1.53393,-0.939336,0.000000,1.223145
...,...,...,...,...,...
595,1.341641,1.53393,0.939336,0.000000,-2.745619
596,1.341641,1.53393,0.939336,1.224745,-3.002517
597,1.341641,1.53393,1.565561,-1.224745,-1.851210
598,1.341641,1.53393,1.565561,0.000000,-3.348623


In [6]:
# https://blog.keras.io/building-autoencoders-in-keras.html
# autoencoder require input to be in same dimensionality, 
# since we have varied dimensions, each will get its own autoencoder, but the hidden layer will have same dimension.

In [7]:
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
encoding_dim = 3
input_img = Input(shape=(4,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(5, activation='sigmoid')(encoded)

# input to its reconstruction 
autoencoder = Model(input_img, decoded)
# input to latent space
encoder = Model(input_img, encoded)

# latent to reconstruction
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')